In [40]:
import matplotlib.pyplot as plt
import pandas as pd
import streamlit as st

from recommend_algo import OptimizeClasses, TopicBasedRecommender


df_grad = pd.read_csv("成績データ_池上.csv", encoding="utf-8")
df_grad = df_grad[df_grad['総合評価']!="履修中"]
recommender = TopicBasedRecommender(df_grad, num_topics=15)
recommender.create_lda_model()
keywords_list = recommender.get_keywords_list()
topic_keywords = recommender.get_topic_keywords(keywords_list)
recommender.assign_info_to_courses()
recommender.assign_topic_to_courses()
number_of_recommendations_by_topic = recommender.decide_number_of_recommendations_by_topic()

#for topic, n_r in number_of_recommendations_by_topic:
for topic in range(recommender.num_topics):
    #if n_r == 0:
    #    continue
    temp, your_course = recommender.execute_recommendation(topic)
    print("-" * 50)
    print(f"トピック {topic}")
    print(f"推定関心度：{recommender.user_profile_percent[topic]}%")
    print("推薦授業: ", end='')
    print("、".join(class_ for class_ in temp['授業科目名'].tolist()))
    print("トピック重要ワード: ", end='')
    print("、".join(word for word in topic_keywords[topic][0]))  # リストをカンマ区切りで表示
    print("トピック専門用語: ", end='')
    print("、".join(keyword for keyword in topic_keywords[topic][1]))  # リストをカンマ区切りで表示
    print("このトピックはあなたが履修した以下の授業に基づいています: ", end='')
    print("、".join(class_ for class_ in your_course))
    print()

df = recommender.df_grad_courses
opt = OptimizeClasses(df)
print(opt.optimize())

c:\Users\sueya\Git-Repositories\monketsu\recommend_algo.py:44: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.df_grad['総合評価'] = self.df_grad['総合評価'].replace(grade_mapping).astype('int')



ユーザーの関心トピック分布: [0.01061693 0.03841711 0.01839078 0.01061693 0.16463217 0.14268258
 0.07792455 0.21133404 0.02984763 0.08245206 0.14561709 0.03561735
 0.01061693 0.01061693 0.01061693]
--------------------------------------------------
トピック 0
推定関心度：1%
推薦授業: 
トピック重要ワード: 人間、工学、科学、インタフェース、ロボット、建築、システム、神経、コンピュータ、インタラクション
トピック専門用語: 工学、人間、コンピュータ、科学、神経、原理、建築、インタフェース、メディア、デザイン、処理、仮想現実、ハードウェア、外界、パターン、高温、物理的
このトピックはあなたが履修した以下の授業に基づいています: 

--------------------------------------------------
トピック 1
推定関心度：3%
推薦授業: 
トピック重要ワード: 政策、経済、分析、統計、数学、国際、金融、モデル、開発、情報
トピック専門用語: 情報、財政、方式、公開鍵暗号、電子署名、実証、現象、ネットワーク、貿易、整数論
このトピックはあなたが履修した以下の授業に基づいています: 

--------------------------------------------------
トピック 2
推定関心度：1%
推薦授業: 
トピック重要ワード: 工学、力学、システム、モデル、制御、原子力、メカニズム、土地、ロボット、動力
トピック専門用語: システム、力学、制御、ロボット、方程式、物理、土地、大規模、動力、構造、周波数、深層学習、分子、結晶、実証分析、日商簿記、シミュレーター、耐震、伝熱工学
このトピックはあなたが履修した以下の授業に基づいています: 

--------------------------------------------------
トピック 3
推定関心度：1%
推薦授業: 
トピック重要ワード: 経営、会計、力学、微積分、理論、解析、デジタル、商学、専門知識、戦略
トピック専

In [41]:
recommender.df_grad_courses.columns

Index(['授業科目名', 'キーワード', '科目番号', '時間割', '単位数', '実施学期', '曜時限', '学位プログラム',
       '科目区分', '科目区分名', 'シラバス', '関連トピック', 'トピック', 'トピック選好', 'トピック確度', '推薦スコア',
       'おすすめ度'],
      dtype='object')

In [51]:
temp = recommender.df_grad_courses[['授業科目名', 'トピック確度', 'トピック', 'おすすめ度']]
temp = temp.rename(columns={'トピック確度': 'トピック比率'})
temp = temp[temp['トピック'].isin(['C', 'H', 'F'])].sort_values(['授業科目名', 'トピック'])
temp

,授業科目名,トピック比率,トピック,おすすめ度
231,システムモデリング,0.597135,C,0
187,システム信頼性解析演習,0.883332,C,0
196,システム制御,0.677195,C,0
195,システム最適化,0.725757,F,10
232,スマートインフォメディアシステム特論,0.866664,F,12
20,ソフトコンピューティング基礎論,0.421728,H,8
235,デジタル制御特論,0.688888,C,0
21,データマイニング,0.866666,H,18
197,データ工学特論I,0.413579,C,0
198,データ工学特論II,0.866664,C,0


In [ ]:
temp = recommender.df_grad_courses[['授業科目名', 'トピック確度', 'トピック', 'おすすめ度']]
temp = temp.rename(columns={'トピック確度': 'トピック比率'})
temp = temp[temp['トピック'].isin(['C', 'H'])].sort_values(['トピック', 'トピック比率'], ascending=False)
temp

In [56]:
temp = recommender.df_grad_courses[['授業科目名', 'トピック確度', 'トピック', 'おすすめ度']]
temp = temp.rename(columns={'トピック確度': 'トピック比率'})
temp = temp[temp['トピック'].isin(['C', 'H'])].sort_values(['トピック', 'トピック比率'], ascending=False)
temp

,授業科目名,トピック比率,トピック,おすすめ度
21,データマイニング,0.866666,H,18
20,ソフトコンピューティング基礎論,0.421728,H,8
238,ロボット制御論,0.883333,C,0
246,自律移動ロボット学,0.883333,C,0
187,システム信頼性解析演習,0.883332,C,0
239,運動制御論,0.866666,C,0
240,音響工学特論,0.866665,C,0
198,データ工学特論II,0.866664,C,0
256,構造物設計法論,0.866664,C,0
251,マイクロメカニックス,0.844441,C,0


In [53]:
temp = recommender.df_grad_courses[['授業科目名', 'トピック確度', 'トピック']]
temp = temp.rename(columns={'トピック確度': 'トピック比率'})
temp = temp[temp['トピック'].isin(['C', 'H'])].sort_values(['トピック', 'トピック比率'], ascending=False)
temp

,授業科目名,トピック比率,トピック
21,データマイニング,0.866666,H
20,ソフトコンピューティング基礎論,0.421728,H
238,ロボット制御論,0.883333,C
246,自律移動ロボット学,0.883333,C
187,システム信頼性解析演習,0.883332,C
239,運動制御論,0.866666,C
240,音響工学特論,0.866665,C
198,データ工学特論II,0.866664,C
256,構造物設計法論,0.866664,C
251,マイクロメカニックス,0.844441,C


In [35]:
recommender.df_grad_courses[['授業科目名', 'トピック', 'おすすめ度']]

,授業科目名,トピック,おすすめ度
0,MDA異分野連携ゼミナール,N,1
1,社会工学のための数学,J,1
2,ミクロ経済学,F,5
3,社会シミュレーション,K,6
4,ゲーム理論,E,5
...,...,...,...
277,実世界指向インタフェース,A,3
278,神経運動制御,C,1
279,触覚の計算論,A,3
280,実験心理学方法論,E,5


In [33]:
recommender.df_grad_courses[['授業科目名', 'トピック', 'おすすめ度', '時間割']].sort_values('トピック確度', ascending=False).head(20).reset_index(inplace=False, drop=True)

KeyError: 'トピック確度'

In [ ]:
word2vecは精度が低く解釈性も微妙

In [ ]:
recommender.df_grad_courses[['授業科目名', 'トピック']].sort_values('トピック確度', ascending=False).head(20).reset_index(inplace=False, drop=True)

In [4]:
!pip install pydantic

     -------------------------------------- 431.4/431.4 kB 9.0 MB/s eta 0:00:00
     ---------------------------------------- 2.0/2.0 MB 15.8 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from pydantic import BaseModel, Field, StrictInt, ValidationError, field_validator, ConfigDict

class MachineConfig(BaseModel):
    model_config = ConfigDict(
        frozen=True,  # Prevents changes to the instance
        extra="forbid",  # No extra fields allowed
        validate_assignment=True,  # Validates values during assignment
    )

    CC_1_UNUSE_START: StrictInt = Field(
        0, description="Start hour for CC-1 non-use period", ge=0, le=23
    )

    CC_1_UNUSE_END: StrictInt = Field(
        10, description="End hour for CC-1 non-use period", ge=0, le=23
    )

    @field_validator("CC_1_UNUSE_END")
    @classmethod
    def validate_end_times(cls, v: int, info) -> int:
        print(f"Validating field: {info.field_name}, value: {v}, data: {info.data}")
        return v

# 正常なインスタンス生成
config = MachineConfig(CC_1_UNUSE_START=8, CC_1_UNUSE_END=10)

# エラーケース
try:
    config = MachineConfig(CC_1_UNUSE_START=8, CC_1_UNUSE_END=8)
except ValidationError as e:
    print(e)


Validating field: CC_1_UNUSE_END, value: 10, data: {'CC_1_UNUSE_START': 8}
Validating field: CC_1_UNUSE_END, value: 8, data: {'CC_1_UNUSE_START': 8}


In [13]:
class MachineConfig(BaseModel):
    model_config = ConfigDict(
        frozen=True,  # Makes the config immutable
        extra="forbid",  # Prevents additional fields
        validate_assignment=True,  # Validates even after initialization
    )

    CC_1_UNUSE_START: StrictInt = Field(
        0, description="Start hour for CC-1 non-use period", ge=0, le=23
    )

    CC_1_UNUSE_END: StrictInt = Field(
        10, description="End hour for CC-1 non-use period ", ge=0, le=23
    )

    AC_1_UNUSE_START: StrictInt = Field(
        0, description="Start hour for AC-1 non-use period", ge=0, le=23
    )

    AC_1_UNUSE_END: StrictInt = Field(
        10, description="End hour for AC-1 non-use period", ge=0, le=23
    )

    @field_validator("CC_1_UNUSE_END", "AC_1_UNUSE_END")
    @classmethod
    def validate_end_times(cls, v: int, info) -> int:
        # Get the corresponding start time based on the field being validated
        field_name = info.field_name
        machine_type = field_name.split("_UNUSE_")[0]  # Gets "CC_1" or "AC_1"
        start_field = f"{machine_type}_UNUSE_START"

        start_time = info.data.get(start_field)
        if start_time is None:
            return v

        if start_time == v:
            raise ValueError(
                f"{machine_type} end time ({v}) cannot be equal to start time ({start_time})"
            )

        return v


# 正常なインスタンス生成
config = MachineConfig(CC_1_UNUSE_START=8, CC_1_UNUSE_END=10)

# エラーケース
try:
    config = MachineConfig(CC_1_UNUSE_START=8, CC_1_UNUSE_END=8)
except ValidationError as e:
    print(e)


1 validation error for MachineConfig
CC_1_UNUSE_END
  Value error, CC_1 end time (8) cannot be equal to start time (8) [type=value_error, input_value=8, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error
